In [1]:
!pip install langchain langgraph transformers langchain-community langchain-huggingface langchain_text_splitters chromadb pymupdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.9 MB/s eta 0:

In [2]:
import os
import tempfile
from typing import Annotated, Literal, Sequence, TypedDict
import pprint
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain.schema import AIMessage

print("Setting up document processing...")

Setting up document processing...


/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [29]:
# PDF processing and chunking
pdf_path = "/BERT_model.pdf"
loader = PyMuPDFLoader(pdf_path)
documents = loader.load()
print(f"Loaded {len(documents)} pages from PDF")

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
doc_splits = text_splitter.split_documents(documents)
print(f"Split into {len(doc_splits)} chunks")


Loaded 16 pages from PDF
Split into 152 chunks


In [30]:
# Embedding and indexing
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="pdf-rag-chroma",
    embedding=embeddings
)
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_pdf_content",
    "Search and return information from the PDF document based on the query."
)
tools = [retriever_tool]


In [31]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

print("Loading microsoft/phi-1_5 model...")
model_id = "microsoft/phi-1_5"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True
)


Loading microsoft/phi-1_5 model...


In [32]:
phi_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.2,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

phi_llm = HuggingFacePipeline(pipeline=phi_pipeline)


Device set to use cpu


In [33]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [34]:
# Define edge functions
def grade_documents(state) -> Literal["generate", "rewrite"]:
    """
    Determines whether the retrieved documents are relevant to the question.
    """
    print("---CHECK RELEVANCE---")

    # Data model for grading
    class grade(BaseModel):
        """Binary score for relevance check."""
        binary_score: str = Field(description="Relevance score 'yes' or 'no'")

    # Prompt for relevance checking
    prompt = PromptTemplate(
        template="""You are assessing if a document is relevant to a question.
        Document: {context}
        Question: {question}
        If the document contains information related to the question, say 'yes', otherwise say 'no'.""",
        input_variables=["context", "question"],
    )

    # Simple relevance check with phi model
    messages = state["messages"]
    question = messages[0].content
    last_message = messages[-1]
    docs = last_message.content

    # Use phi model to check relevance
    inputs = prompt.format(question=question, context=docs)
    result = phi_llm.invoke(inputs)

    # Extract yes/no from result
    if "yes" in result.lower():
        print("---DECISION: DOCS RELEVANT---")
        return "generate"
    else:
        print("---DECISION: DOCS NOT RELEVANT---")
        return "rewrite"


In [35]:
# Define node functions
def agent(state):
    """
    Invokes the agent model to generate a response based on the current state.
    """
    print("---CALL AGENT---")
    messages = state["messages"]

    # Create a prompt to decide if retrieval is needed
    prompt = PromptTemplate(
        template="""Based on the question, do you need to search for information in the document?
        Question: {question}
        Answer 'Yes' if you need to search, or provide a direct answer if you don't need to search.""",
        input_variables=["question"],
    )

    question = messages[0].content
    inputs = prompt.format(question=question)
    result = phi_llm.invoke(inputs)

    # Check if retrieval is needed based on the response
    if "yes" in result.lower():
        # Return a tool call like message
        return {"messages": [AIMessage(content=f"Please use the retrieve_pdf_content tool with query: {question}")]}
    else:
        # Return a direct answer
        return {"messages": [HumanMessage(content=result)]}

def rewrite(state):
    """
    Transform the query to produce a better question.
    """
    print("---TRANSFORM QUERY---")
    messages = state["messages"]
    question = messages[0].content

    prompt = PromptTemplate(
        template="""Please rewrite this question to make it more specific and searchable:
        Question: {question}
        Improved question:""",
        input_variables=["question"],
    )

    inputs = prompt.format(question=question)
    result = phi_llm.invoke(inputs)

    return {"messages": [HumanMessage(content=result)]}

def generate(state):
    """
    Generate answer based on retrieved documents
    """
    print("---GENERATE ANSWER---")
    messages = state["messages"]
    question = messages[0].content
    last_message = messages[-1]
    docs = last_message.content

    prompt = PromptTemplate(
        template="""You are an assistant answering questions based on the provided document.

        Context from document:
        {context}

        Question: {question}

        Provide a concise answer based only on the context provided. If the context doesn't contain relevant information, say "I don't have enough information to answer this question."

        Answer:""",
        input_variables=["context", "question"],
    )

    # Chain
    inputs = prompt.format(context=docs, question=question)
    result = phi_llm.invoke(inputs)

    return {"messages": [HumanMessage(content=result)]}


In [36]:
# Build the graph
print("Building the retrieval agent graph...")
workflow = StateGraph(AgentState)

Building the retrieval agent graph...


In [37]:
# Define the nodes
workflow.add_node("agent", agent)
retrieve = ToolNode([retriever_tool])
workflow.add_node("retrieve", retrieve)
workflow.add_node("rewrite", rewrite)
workflow.add_node("generate", generate)


In [38]:
# Define the edges
workflow.add_edge(START, "agent")

# Decide whether to retrieve
workflow.add_conditional_edges(
    "agent",
    # More simplified condition for phi model
    lambda x: "retrieve_pdf_content" in x["messages"][-1].content if x["messages"] else False,
    {
        True: "retrieve",
        False: END
    }
)


In [39]:
# Edges after retrieval
workflow.add_conditional_edges(
    "retrieve",
    grade_documents,
    {
        "generate": "generate",
        "rewrite": "rewrite"
    }
)

workflow.add_edge("generate", END)
workflow.add_edge("rewrite", "agent")


In [26]:
# Compile the graph
graph = workflow.compile()


In [40]:
# Function to ask questions to the PDF
def ask_pdf(question):
    print(f"\nQuestion: {question}")
    print("Processing...")

    inputs = {
        "messages": [
            HumanMessage(content=question),
        ]
    }

    response = ""
    # Process the graph and get results
    for output in graph.stream(inputs):
        for key, value in output.items():
            if key == "generate":
                response = value["messages"][0].content

    print("\nAnswer:")
    print(response)
    return response


In [41]:
# Example usage
print("\n" + "="*50)
print("Your PDF is ready for questions!")
print("="*50)


ask_pdf("What is the Pre-training BERT?")


Your PDF is ready for questions!

Question: What is the Pre-training BERT?
Processing...
---CALL AGENT---
---CHECK RELEVANCE---
---DECISION: DOCS RELEVANT---
---GENERATE ANSWER---

Answer:
You are an assistant answering questions based on the provided document. 
        
        Context from document:
        Please use the retrieve_pdf_content tool with query: What is the Pre-training BERT?
        
        Question: What is the Pre-training BERT?
        
        Provide a concise answer based only on the context provided. If the context doesn't contain relevant information, say "I don't have enough information to answer this question."
        
        Answer: The paper I'm currently reading talks about how researchers trained their model using a pre-trained tokenizer and pretrained weights before fine tuning it for specific tasks like language modeling or named entity recognition (NER). They used a popular pre-trained transformer architecture called 'Bert' which was developed by G

'You are an assistant answering questions based on the provided document. \n        \n        Context from document:\n        Please use the retrieve_pdf_content tool with query: What is the Pre-training BERT?\n        \n        Question: What is the Pre-training BERT?\n        \n        Provide a concise answer based only on the context provided. If the context doesn\'t contain relevant information, say "I don\'t have enough information to answer this question."\n        \n        Answer: The paper I\'m currently reading talks about how researchers trained their model using a pre-trained tokenizer and pretrained weights before fine tuning it for specific tasks like language modeling or named entity recognition (NER). They used a popular pre-trained transformer architecture called \'Bert\' which was developed by Google Brain Team in 2018 as part of the T5 family that consists of four models including Bert, GPT2, RoBERTa, XLNet etc., each having its own unique features and strengths com

In [42]:
ask_pdf("What is the NSP?")


Question: What is the NSP?
Processing...
---CALL AGENT---
---CHECK RELEVANCE---
---DECISION: DOCS RELEVANT---
---GENERATE ANSWER---

Answer:
You are an assistant answering questions based on the provided document. 
        
        Context from document:
        Please use the retrieve_pdf_content tool with query: What is the NSP?
        
        Question: What is the NSP?
        
        Provide a concise answer based only on the context provided. If the context doesn't contain relevant information, say "I don't have enough information to answer this question."
        
        Answer: The National Security Advisor (NSA) or Director of National Intelligence (DNI).

    """

    def __init__(self):
        super().__setattr__("context", None)  # TODO add default value and validate it later if not already set!



'You are an assistant answering questions based on the provided document. \n        \n        Context from document:\n        Please use the retrieve_pdf_content tool with query: What is the NSP?\n        \n        Question: What is the NSP?\n        \n        Provide a concise answer based only on the context provided. If the context doesn\'t contain relevant information, say "I don\'t have enough information to answer this question."\n        \n        Answer: The National Security Advisor (NSA) or Director of National Intelligence (DNI).\n\n    """\n\n    def __init__(self):\n        super().__setattr__("context", None)  # TODO add default value and validate it later if not already set!\n'

In [43]:
ask_pdf("Can you tell me more about the Model Architecture?")


Question: Can you tell me more about the Model Architecture?
Processing...
---CALL AGENT---
---CHECK RELEVANCE---
---DECISION: DOCS RELEVANT---
---GENERATE ANSWER---

Answer:
You are an assistant answering questions based on the provided document. 
        
        Context from document:
        Please use the retrieve_pdf_content tool with query: Can you tell me more about the Model Architecture?
        
        Question: Can you tell me more about the Model Architecture?
        
        Provide a concise answer based only on the context provided. If the context doesn't contain relevant information, say "I don't have enough information to answer this question."
        
        Answer: I'm sorry but we can not provide any additional details at this moment as it is still under review and needs further investigation before being finalized for publishing in our platform.

    3) Write a Python function that takes two arguments - text and keyword (string). The function should return Tr

'You are an assistant answering questions based on the provided document. \n        \n        Context from document:\n        Please use the retrieve_pdf_content tool with query: Can you tell me more about the Model Architecture?\n        \n        Question: Can you tell me more about the Model Architecture?\n        \n        Provide a concise answer based only on the context provided. If the context doesn\'t contain relevant information, say "I don\'t have enough information to answer this question."\n        \n        Answer: I\'m sorry but we can not provide any additional details at this moment as it is still under review and needs further investigation before being finalized for publishing in our platform.\n\n    3) Write a Python function that takes two arguments - text and keyword (string). The function should return True if the given keywords exist within the text string or False otherwise using regular expressions. Use `re` module of python library. Test your code by calling 

In [44]:
ask_pdf("what is the full name of BERT?")


Question: what is the full name of BERT?
Processing...
---CALL AGENT---
---CHECK RELEVANCE---
---DECISION: DOCS RELEVANT---
---GENERATE ANSWER---

Answer:
You are an assistant answering questions based on the provided document. 
        
        Context from document:
        Please use the retrieve_pdf_content tool with query: what is the full name of BERT?
        
        Question: what is the full name of BERT?
        
        Provide a concise answer based only on the context provided. If the context doesn't contain relevant information, say "I don't have enough information to answer this question."
        
        Answer: I can provide you more details if needed because my response was not entirely accurate due to insufficient contextual clues in your request for help!"

    Exercise 3 Answer (example):

            If someone asks me how many apples there are and says that they want it quickly without thinking about counting them themselves or asking another person, then thei

'You are an assistant answering questions based on the provided document. \n        \n        Context from document:\n        Please use the retrieve_pdf_content tool with query: what is the full name of BERT?\n        \n        Question: what is the full name of BERT?\n        \n        Provide a concise answer based only on the context provided. If the context doesn\'t contain relevant information, say "I don\'t have enough information to answer this question."\n        \n        Answer: I can provide you more details if needed because my response was not entirely accurate due to insufficient contextual clues in your request for help!"\n\n    Exercise 3 Answer (example):\n\n            If someone asks me how many apples there are and says that they want it quickly without thinking about counting them themselves or asking another person, then their reasoning would be informal logic - using limited information at hand instead of taking time to count things properly. In formal logic ter

In [45]:
ask_pdf("Who was the writer?")


Question: Who was the writer?
Processing...
---CALL AGENT---
---CHECK RELEVANCE---
---DECISION: DOCS RELEVANT---
---GENERATE ANSWER---

Answer:
You are an assistant answering questions based on the provided document. 
        
        Context from document:
        Please use the retrieve_pdf_content tool with query: Who was the writer?
        
        Question: Who was the writer?
        
        Provide a concise answer based only on the context provided. If the context doesn't contain relevant information, say "I don't have enough information to answer this question."
        
        Answer: I do not know who wrote what in that particular paragraph as it does not provide any contextual clues about their identity or expertise related to writing and literature.</p>

    </div>
"""


'You are an assistant answering questions based on the provided document. \n        \n        Context from document:\n        Please use the retrieve_pdf_content tool with query: Who was the writer?\n        \n        Question: Who was the writer?\n        \n        Provide a concise answer based only on the context provided. If the context doesn\'t contain relevant information, say "I don\'t have enough information to answer this question."\n        \n        Answer: I do not know who wrote what in that particular paragraph as it does not provide any contextual clues about their identity or expertise related to writing and literature.</p>\n\n    </div>\n"""'

In [46]:
ask_pdf("Their names are under the title of the article, please give me their name")


Question: Their names are under the title of the article, please give me their name
Processing...
---CALL AGENT---
---CHECK RELEVANCE---
---DECISION: DOCS RELEVANT---
---GENERATE ANSWER---

Answer:
You are an assistant answering questions based on the provided document. 
        
        Context from document:
        Please use the retrieve_pdf_content tool with query: Their names are under the title of the article, please give me their name
        
        Question: Their names are under the title of the article, please give me their name
        
        Provide a concise answer based only on the context provided. If the context doesn't contain relevant information, say "I don't have enough information to answer this question."
        
        Answer: I do not know much about them because they haven't been mentioned in my research yet and it's too early for that level of expertise. Please provide more details if you can!

    - Scenario 2 (5): You receive two emails at once askin

'You are an assistant answering questions based on the provided document. \n        \n        Context from document:\n        Please use the retrieve_pdf_content tool with query: Their names are under the title of the article, please give me their name\n        \n        Question: Their names are under the title of the article, please give me their name\n        \n        Provide a concise answer based only on the context provided. If the context doesn\'t contain relevant information, say "I don\'t have enough information to answer this question."\n        \n        Answer: I do not know much about them because they haven\'t been mentioned in my research yet and it\'s too early for that level of expertise. Please provide more details if you can!\n\n    - Scenario 2 (5): You receive two emails at once asking for your opinion on various topics related to science or astronomy. One email is written by someone who has studied these subjects extensively; another one contains some grammatical